In [3]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Introduction à `Scikit-learn`

Dans ce notebook, il est question de fournir une introduction à la bibliothèque populaire d'apprentissage automatique `Scikit-learn` qui fournit de nombreux modèles prêts à l'emploi pour faire de l'apprenstissage automatique tout en rendant souple le processus de création de combinaison de plusieurs modèles. 

En résumé, l'apprensitssage automatique (Machine learning : ML) vise à construire une fonction $f$ qui étant donné un vecteur de **caractéristiques (features en anglais)** $X$, de contruire un lien avec la sortie $y$ encore appelé **étiquete (label en anglais)**, en d'autres termes le ML chercher une fonction $f$ tel que $f(X) \approx y$. Lorsque $y$ est connu, on parle **d'apprensitssage supervisé** et lorsque $y$ est inconnu, on parle **d'apprensitasse non supervisé**. La fonction $f$ est le modèle que l'on cherche à établir. Il existe plusieurs modèles en fonction du type de problèmes et même des données que l'on dispose. Il y'a tout d'abord souvent une phase dite **d'apprentissage** pendant laquel le modèle est constuit avec un sous-ensemble de données (avec étiquete, on parle de données étiquetées) issu des données initiales et une phase dite de **test** où le modèle consruit est testé avec un autre sous-ensemble de données.

Pour cet exemple introducif sur `scikit-learn`, nous allons travailler avec des données sur le prix médian des maisons (regroupées par groupes) dans l'état de Californie aux états unis.

In [1]:
from sklearn.datasets import fetch_california_housing

# On télécharge les données
data = fetch_california_housing()
X = data['data']
y = data['target']

print(data['DESCR']) # Description du jeu de données (dataset)

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [49]:
y[0:3]

array([4.526, 3.585, 3.521])

En revanche, chaque modèle possède ses propres paramètres souvent appelés **hyper-paramètres**. `Scikit-learn` les utilise dans le processus d'apprentissage pour calibrer le modèle par rapport au problème en face. Dans l'exemple suivant, on a crée le modèle de regression de **Ridge** qui est un type de regression multiple avec un **hyper-paramètre**. 

In [26]:
from sklearn.linear_model import Ridge 

ridge = Ridge(alpha=0.1) # alpha est un hyperparamètre initialisé à 0.1

In [41]:
Ridge?

Dans la bibliothèque `scikit-learn`, chaque modèle de ML est vu comme une classe à part entière avec ses propres **hyper-paramètres** le plus souvent avec des bonnes valeurs par défauts. ils sont appelés des **estimateurs** (**estimators**), il en existe **trois** types : **classifieurs (classifiers), regresseurs (regressors), et transformateurs (transformers)** tous héritant de la classe estimateur de base dénomée `BaseEstimator`. En plus de cette classe de Base, chaque type d'estimateur hérite d'une autre classe selon son type, c'est-à-dire soit `RegressorMixin` (pour un regresseur) soit `ClassifierMixin` (pour un classifieur) soit `TransformerMixin` (pour un transformateur). Ainsi si on souhaite écrire un nouveau modèle il faudra créer une classe qui hérite de `BaseEstimator` et de la classe basique correspondante par exemple `RegressorMixin` si on souhaite écrire un nouveau regresseur.

Tous les estimateurs (algorithmes d'apprenstissage) peuvent être classés en deux groupes : 

1. **Prédicteurs (predictors)**
2. **Transformateurs (transformers)**

## Prédicteurs : classifieurs et regresseurs

Comme leurs noms l'indiquent, ils vont faire de la prédiction, soit de la classification (dans ce cas, le label n'est pas un réel) ou de la régression (le label est un réel). Ces classes possèdent deux méthodes principales : 

* `fit(X, y)`: entraine ou ajuste l'objet (ici le modèle) à la matrice de caractéristiques $X$ et le label $y$.
* `predict(X)`: fait des prédictions sur l'ensemble des données passées $X$.

In [12]:
from sklearn.linear_model import LinearRegression # Importation du modèle de regression linéaire

# On crée le modèle et l'entraine/ajuste
model = LinearRegression()
model.fit(X, y)

# On prédit les étiquetes des caractéristiques X
y_pred = model.predict(X)

print(y_pred)
print("Dimension du tableau de prédiction: {}".format(y_pred.shape))
print("Dimension de l'ensemble d'apprensissage: {}".format(X.shape))

[4.13164983 3.97660644 3.67657094 ... 0.17125141 0.31910524 0.51580363]
Dimension du tableau de prédiction: (20640,)
Dimension de l'ensemble d'apprensissage: (20640, 8)


In [42]:
LinearRegression?

Étant donné que dans ce cas, nous avons un vecteur des caractéristiques de taille $8$, et nous utilisons le modèle regression linéaire, alors le modèle s'écrit  de la façon suivante : 
$$ y(X) = \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 + \beta_4 x_4 + \beta_5 x_5 + \beta_6 x_6 + \beta_7 x_7 + \beta_8 x_8 + \beta_0. $$

Après apprensissage (ajustement), `scikit-learn` stocke les coefficients dans des **attributs spécifiques** du modèle avec un _underscore_, et on peut les recupérer dans les attributs `coefs_` et `intercept_` du modèle (voir aide sur le modèle de regression linéaire `LinearRegression`).

In [43]:
# Ici, on peut consulter les hyper-paramètres de ce modèle.
print("β_0: {}".format(model.intercept_)) 

for i in range(8):
    print("β_{}: {}".format(i+1, model.coef_[i]))

β_0: -36.941920207184516
β_1: 0.4366932931343246
β_2: 0.009435778033238069
β_3: -0.10732204139090418
β_4: 0.6450656935198134
β_5: -3.976389421207444e-06
β_6: -0.00378654265497081
β_7: -0.42131437752714374
β_8: -0.43451375467477804


Il est souhaitable de savoir si oui ou non le modèle prédit bien, nous pouvons faire appel à la méthode `score(X, y)` dont hérite par défaut tout prédciteur, elle fonctionne en deux étapes : 
1. Elle exécute en premier la méthode `predict(X)` pour déterminer les valeurs prédites de $X$ par le modèle
2. Utilise ces valeurs prédites pour évaluer le modèle par rapport aux valeurs ($y$) qui lui sont passées en paramètres.

Il faut savoir que cette évaluation est fonction du type de modèle soit une regression ou une classification. Pour une regression, c'est la valeur du $R^2$ qui est utilisé et pour une classification, c'est la précision. le R-carré ou $R^2$ est utilisé en statistique pour juger de la qualité d'une regression linéaire.

Pour $n$ valeurs à prédire $\hat y_i, i=1\cdots n$ de $y_i$, $R^2$ est défini par : 
$$R^2 = 1-\dfrac{\sum_{i=1}^n\left(y_i-\hat{y_i}\right)^2}{\sum_{i=1}^n\left(y_i-\bar y\right)^2}$$

On peut tout aussi utiliser d'autres métriques comme le **Root Mean Square Error (RMSE)** défini par : 
**RMSE** = $\sqrt{\frac{1}{n}\sum_{i=1}^{n}\,(y_i - \hat y^i)^2}$

In [44]:
model.score?

In [23]:
print("R^2: {:g}".format(model.score(X, y)))

R^2: 0.606233


La valeur du score nous donne un indice pour savoir si on peut faire appel à d'autres types de modèles plus adaptés à la situation . Mais avant de les utiliser, il faut les connaitre, et savoir leurs limites surtout en terme de temps de calcul, il est toutefois possible de combiner plusieurs modèles. 

Voici un exemple de modèle plus adapté à cette situation que la regression linéaire.

In [45]:
from sklearn.ensemble import GradientBoostingRegressor # Regression linéaire utilisant la descente du gradient

# On crée le modèle et l'entraine/ajuste
model = GradientBoostingRegressor()
model.fit(X, y)

# On prédit les étiquetes des caractéristiques X
y_pred = model.predict(X)

print(y_pred)
print("R^2: {:g}".format(model.score(X, y)))

[4.26432728 3.87864519 3.92074556 ... 0.63664692 0.74759279 0.7994969 ]
R^2: 0.803324


Ce modèle est bien meilleur que la regression simple dans ce cas.

## Transformateurs

Ce sont des modèles qui agissent sur les données dans le but de les transformer pour usage ultérieur. Ils sont très utiles par exemple pour mettre à l'échelle un ensemble de données car certains algoruthmes de ML ne fonctionnent correctement que lorsque les données sont mises à l'échelle, par exemple centrées, c'est le cas des technique d'analyse comme **l'analyse en composante principale (ACP)**. Les modèles dans cette classe implémentent les interfaces suivantes : 

* `fit(X)`: entraine ou ajuste l'objet (le modèle) à la matrice de caractéristiques $X$.
* `transform(X)`: applique une transformation sur la matrice de caractéristiques $X$ en utilisant tous les paramètres appris
* `fit_transform(X)`: applique la méthode `fit(X)` ensuite `transform(X)`.

`StandardScaler` est un transformateur qui permet de centrer les données de manière à avoir une moyenne $0$ et une variance égale à $1$. Formelleemnt, il transforme la cractéristique $x_i$ en $x'_i$ de la manière suivante : 

$$ x'_i = \frac{x_i - \mu_i}{\sigma_i}. $$

On peut par exemple l'utiliser pour mettre à l'échelle les données des prix des maisons qu'on a chargé plus haut.

In [46]:
from sklearn.preprocessing import StandardScaler

# On crée et ajuste le transformateur
scaler = StandardScaler()
scaler.fit(X)

# On met à l'échelle les données
Xt = scaler.transform(X)

# On crée un data frame des résultats
stats = np.vstack((X.mean(axis=0), X.var(axis=0), Xt.mean(axis=0), Xt.var(axis=0))).T
feature_names = data['feature_names']
columns = ['moyenne non à l\'échelle', 'variance non à l\'échelle', 'moyenne à l\'échelle',
           'variance à l\'échelle']

df = pd.DataFrame(stats, index=feature_names, columns=columns)
df

,moyenne non à l'échelle,variance non à l'échelle,moyenne à l'échelle,variance à l'échelle
MedInc,3.870671,3.609148e+00,6.609700e-17,1.0
HouseAge,28.639486,1.583886e+02,5.508083e-18,1.0
AveRooms,5.429000,6.121236e+00,6.609700e-17,1.0
AveBedrms,1.096675,2.245806e-01,-1.060306e-16,1.0
Population,1425.476744,1.282408e+06,-1.101617e-17,1.0
AveOccup,3.070655,1.078648e+02,3.442552e-18,1.0
Latitude,35.631861,4.562072e+00,-1.079584e-15,1.0
Longitude,-119.569704,4.013945e+00,-8.526513e-15,1.0


## Transformateur Colonne

Les caractéristiques sont souvent de divers types, numériques, catégorielle et même textuelles et pour chaque type de caractéristiques, on applique un transformateur distinct. Le transformateur [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) de `scikit-learn` permet d'appliquer un transformateur par colonne.

In [48]:
from sklearn.compose import ColumnTransformer

col_transformer = ColumnTransformer(
    remainder='passthrough', # On ne transforme pas les autres colonnes (Latitude et Longitude). 'drop' supprime
    transformers=[
        ('scaler', StandardScaler(), slice(0,6)) # les 6 première colonnes
    ]
)

col_transformer.fit(X)
Xt = col_transformer.transform(X)

print('Moyenne MedInc avant transformation?', X.mean(axis=0)[0])
print('Moyenne MedInc après transformation?', Xt.mean(axis=0)[0], '\n')

print('Moyenne Longitude avant transformation?', X.mean(axis=0)[-1])
print('Moyenne Longitude après transformation?', Xt.mean(axis=0)[-1])

Moyenne MedInc avant transformation? 3.8706710029069766
Moyenne MedInc après transformation? 6.609699867535816e-17 

Moyenne Longitude avant transformation? -119.56970445736432
Moyenne Longitude après transformation? -119.56970445736432


Le code suivant permet de supprimer la colonne à l'indice $0$ `'MedInc'`. Celà arrive par exemple lorsque la colonne présente des données corrompues.

In [49]:
col_transformer = ColumnTransformer(
    remainder='passthrough',
    transformers=[
        ('remove', 'drop', 0), # On supprime la première colonne
        ('scaler', StandardScaler(), slice(1,6)) # On applique la transformation sur le vecteur les colonnes 1 à 6
    ]
)

Xt = col_transformer.fit_transform(X)

print('Nombre de caractéristiques dans X:', X.shape[1])
print('Nombre de caractéristiques dans Xt:', Xt.shape[1])

Nombre de caractéristiques dans X: 8
Nombre de caractéristiques dans Xt: 7


## Pipelines

Il arrive que l'on soit amener à construire un mélange de modèles pour aboutir à un modèle plus complexe et plus amélioré quoique parfois coûteux en terme de temps d'apprenstissage et même de prévision. `Scikit-learn` réalise celà grace à la classe `Pipeline` qui est un estimateur caractérisé par une succession de transformateurs avec au final un estimateur.
Dans l'exemple qui suit, on va : 

1. Mettre à l'échelle le jeu de données.
1. Ajouter des caractéristiques polynomiales.
1. Entrainer un modèle linéaire de regression avec le jeu transformé.

In [51]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

# on construit le pipeline
scaler = StandardScaler()
poly_features = PolynomialFeatures(degree=2)
lin_reg = LinearRegression()

pipe = Pipeline([
    ('scaler', scaler),
    ('poly', poly_features),
    ('regressor', lin_reg)
])

In [41]:
pipe.named_steps # Dictonnaire donnant la liste des transformateurs et estimateurs du pipeline

{'scaler': StandardScaler(copy=True, with_mean=True, with_std=True),
 'poly': PolynomialFeatures(degree=2, include_bias=True, interaction_only=False),
 'regressor': LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False)}

Lorsque nous faisons `pipe.fit(X, y)`, l'ensemble des instruction exécutées est le suivant : 
```
Xt = scaler.fit_transform(X) 
Xt = poly.fit_transform(Xt)
lin_reg.fit(Xt,y)
```
Et lorsqu'on exécute `pipe.predict(X, y)`, l'ensemble des données `X` va passer par tous les transformateurs et à la fin le prédicteur.
```
Xt = scaler.transform(X)
Xt = poly.transform(Xt)
y_pred = lin_reg.predict(Xt)
```

In [42]:
# Entraine/ajuste le modèle et prédit les étiquetes
pipe.fit(X, y)
y_pred = pipe.predict(X)

print(y_pred)
print("R^2: {}".format(pipe.score(X, y)))

[4.00298901 3.92349228 3.99012926 ... 0.83369975 0.88801566 0.97559649]
R^2: 0.6832976293317492


## Union de caractéristiques (Feature Union)

De même qu'il soit capable de combiner plusieurs estimateurs, il est aussi possible de combiner plusieurs transformateurs. `ColumnTransformer` agit sur des colonnes séparemment, mais est limité lorsqu'on souhaite appliquer plusieurs transformateurs sur une colonne précise. `Scikit-learn` fourni la classe `FeatureUnion` qui permet de combiner un ensemble de transformateurs. Il les exécutent en parallèle pour avoir en sortie une seule matrice de données transformées. Avec celà, `scikit-learn` permet de construire des modèles de ML plus complexes.

Pour illustrer le déroulé de `FeatureUnion`, nous allons appliquer deux transformateurs en parallèle afin de produire des nouvelles données qui seront ensuite passées à un modèle de regression linéaire. La méthdode ACP permet une méthode d'analyse de données qui permet la réduction de la dimension des données de grandes tailles (plusieurs caractéristiques)souvent correlées pour produire de nouvelles caractéristiques qui cette-fois seront non correllées. Elle vise à extraire parmis les caractéristiques $X$, celles qui sont les plus importantes pour générer tous les individus. Cette méthode fonctionne bien lorsque les données sont mises à l'échelle. La fonction `PCA` de la bibliothèque permet de la mettre en exergue.

`SelectKBest` renvoie les $k$ meilleurs caractéristiques qui vérifient un certain critère. Par exemple, on peut rechercher les $3$ meilleurs caractéristiques fortement correlées avec l'étiquete (label). `VarianceThreshold` renvoie les caractéristiques qui ont une variance supérieure à un seuil fixé, on va donc combiner les deux transformateurs pour créer un nouveau jeu de caractéristiques. 

In [66]:
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import FeatureUnion

selector = SelectKBest(f_regression, k=3)
var_sel = VarianceThreshold(threshold = 1)

union = FeatureUnion([('var_sel', var_sel), ('selector', selector)])
pipe = Pipeline([('union', union), ('regressor', lin_reg)])
pipe.fit(X, y)

print("nombre de colonne/caractéristique dans le jeu originel: {}".format(X.shape[-1]))
print("nombre de colonne/caractéristique dans le nouveau jeu: {}".format(union.transform(X).shape[-1]))
print("R^2: {}".format(pipe.score(X, y)))

nombre de colonne/caractéristique dans le jeu originel: 8
nombre de colonne/caractéristique dans le nouveau jeu: 10
R^2: 0.5961995839710024


On a donc au final 10 variables dont $3$ ont une meilleure correlation avec la label, ça veut dire que `VarianceThreshold` a éliminé une variable et produit $7$.